In [1]:
%pip install git+https://github.com/m-bain/whisperx.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-krz9t7sc
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-krz9t7sc
  Resolved https://github.com/m-bain/whisperx.git to commit 9e3a9e0e38fcec1304e1784381059a0e2c670be5
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
%pip install pyannote.audio pydub python-docx requests

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import os
from pathlib import Path
import pandas as pd
from pyannote.audio import Pipeline
import whisperx
import pydub
import subprocess
import time
import requests
import json
import torch
from docx import Document

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-09-08 03:03:58.797077: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def load_pipeline_from_pretrained(path_to_config: str | Path) -> Pipeline:
    path_to_config = Path(path_to_config)
    print(f"Loading pyannote pipeline from {path_to_config}...")
    pipeline = Pipeline.from_pretrained(path_to_config)
    return pipeline


In [5]:
def convert_to_wav(input_file: str, output_file: str):
    print(f"Конвертация {input_file} в WAV")
    with open(os.devnull, 'w') as devnull:
        subprocess.call(['ffmpeg', '-i', input_file, output_file], stdout=devnull, stderr=devnull)


In [6]:
def process_audio_file(audio_file: str, pipeline, whisper_model, device="cuda"):
    file_name = Path(audio_file).stem
    wav_file = f"{file_name}.wav"
    
    # Конвертация в WAV, если нужно
    if not audio_file.endswith(".wav"):
        convert_to_wav(audio_file, wav_file)
    else:
        wav_file = audio_file

    # Диаризация
    print(f"Диаризация {wav_file}")
    diarization_result = pipeline({"audio": wav_file})
    
    diarization_list = []
    for segment, track, label in diarization_result.itertracks(yield_label=True):
        diarization_list.append({
            'start': segment.start,
            'end': segment.end,
            'speaker': label
        })

    diarization_df = pd.DataFrame(diarization_list)

    # Транскрипция с замером времени
    print(f"Транскрипция {wav_file}")
    start_time = time.time()
    transcription_result = whisper_model.transcribe(wav_file)
    end_time = time.time()

    transcription_time = end_time - start_time
    print(f"Время транскрипции: {transcription_time:.2f} секунд")

    # Выравнивание сегментов
    aligned_model, metadata = whisperx.load_align_model(
        language_code=transcription_result["language"], device=device
    )
    aligned_result = whisperx.align(
        transcription_result["segments"], aligned_model, metadata, wav_file, device
    )

    # Присвоение говорящих сегментам
    segments_with_speakers = whisperx.assign_word_speakers(diarization_df, aligned_result)
    
    all_transcription_list = []
    for segment in segments_with_speakers['segments']:
        if isinstance(segment, dict):
            speaker = segment.get('speaker', 'unknown')
            text = segment.get('text', '')
            all_transcription_list.append({
                'speaker': speaker,
                'text': text
            })

    transcription_df = pd.DataFrame(all_transcription_list)
    
    # Подсчет уникальных спикеров
    unique_speakers = diarization_df['speaker'].unique()
    num_speakers = len(unique_speakers)
    
    # Конкатенация всех текстов
    full_text = ' '.join(transcription_df['text'].tolist())
    
    return file_name, full_text, num_speakers, transcription_time

In [7]:
def process_text_with_yandex_gpt(full_text: str, token: str):
    url = 'https://llm.api.cloud.yandex.net/foundationModels/v1/completion'
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    
    # Текст для отправки в Yandex GPT
    prompt = """
Текст в таком формате: 
(время в минутах) текст

ожалуйста, поработайте в качестве виртуального секретаря, обрабатывая и отвечая на запрос, как если бы вы были настоящим административным помощником. Мне бы хотелось, чтобы вы делали заметки, отвечали на вопросы, назначали встречи и готовили корреспонденцию профессионально и своевременно. Приступайте к выполнению своих секретарских обязанностей, давая четкие и лаконичные ответы, как будто мы находимся на деловой встрече или разговариваем по телефону». Эта модифицированная подсказка проясняет ожидаемое поведение модели и обеспечивает контекст для ее работы в качестве виртуального секретаря, что должно привести к более точным и конкретным ответам.

Задание:

Вам предоставляется текст совещания. Ваша задача — максимально извлечь информацию и представить её строго по следующему шаблону.

ТЕМА ВСТРЕЧИ
ПРОТОКОЛ ВСТРЕЧИ

Дата: Дата проведения встречи (если не указана, пропустите этот пункт).
Время: Время начала и окончания совещания (если неизвестно, пропустите).
Длительность: Длительность совещания.
Участники: Список всех участников совещания, определи их по тексту
ПОВЕСТКА ДНЯ
Цель встречи: Краткое описание цели встречи, определи по тексту
I. Название вопроса 1
II. Название вопроса 2 (если обсуждался, иначе пропустите)

РЕЗЮМЕ ОБСУЖДЕНИЙ
I. Название вопроса 1:
a) Докладчики: Имя (если есть)

Решено: Описание решения (если принято).
Срок: Срок выполнения решения (если указан).
Ответственный: ФИО ответственного (если есть).
Образ результата: Описание ожидаемого результата (если указано).
Контекст обсуждения: Описание контекста обсуждения, которое привело к принятому решению. В скобках не указывай время
Время: Время в аудиозаписи (мм:сс).
II. Название вопроса 2 (если обсуждался):
a) Докладчики: Имя

Решено: Описание решения (если принято).
Контекст обсуждения: Краткое описание контекста обсуждения.
Время: Время в аудиозаписи (мм:сс).

Важно: Если какие-либо блоки (например, второе обсуждение) не были упомянуты в совещании, просто пропустите их, не выводите "не обсуждалось" или аналогичные пометки. Но разбиение на блоки текст - главная ваша задача. Время указывать обязательно в ответе должны быть такие слова как 
ТЕМА ВСТРЕЧИ: ПРОТОКОЛ ВСТРЕЧИ Дата: Время: Длительность: Участники: ПОВЕСТКА ДНЯ Цель встречи: РЕЗЮМЕ ОБСУЖДЕНИЙ
Ответ должен быть максимально развернутым, 
"""
    
    data = {
        "modelUri": "gpt://b1g72uajlds114mlufqi/yandexgpt/latest",
        "completionOptions": {
            "stream": False,
            "temperature": 0.6,
            "maxTokens": 2000
        },
        "messages": [
            {
                "role": "system",
                "text": prompt
            },
            {
                "role": "user",
                "text": full_text
            }
        ]
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        response_json = response.json()
        return response_json['result']['alternatives'][0]['message']['text']
    else:
        print(f"Ошибка: {response.status_code}")
        return None


In [8]:
def create_doc_from_text(text, filename):
    start_time = time.time()
    
    doc = Document()

    # Разбиваем текст на строки
    lines = text.strip().split("\n")

    # Пройдем по строкам и добавим форматирование
    for line in lines:
        line = line.strip()
        if line.startswith("ТЕМА ВСТРЕЧИ:") or line.startswith("**Тема встречи:**"):
            # Добавляем заголовок темы встречи
            doc.add_heading(line, level=1)
        elif line.startswith("ПРОТОКОЛ ВСТРЕЧИ") or line.startswith("**Протокол встречи:**"):
            doc.add_heading("Протокол встречи", level=2)
        elif line.startswith("Дата:") or line.startswith("* **Дата:**"):
            # Добавляем дату
            doc.add_paragraph(line, style='Normal')
        elif line.startswith("Время:") or line.startswith("* **Время начала и окончания совещания:**"):
            # Добавляем время
            doc.add_paragraph(line, style='Normal')
        elif line.startswith("Длительность:") or line.startswith("* **Длительность:**"):
            # Добавляем длительность
            doc.add_paragraph(line, style='Normal')
        elif line.startswith("Участники:") or line.startswith("* **Участники:**"):
            doc.add_heading("Участники", level=3)
        elif line.startswith("* ") or line.startswith("    *"):
            # Добавляем список участников
            doc.add_paragraph(line.replace("* ", "").replace("    *", ""), style='List Bullet')
        elif line.startswith("ПОВЕСТКА ДНЯ") or line.startswith("**Повестка дня:**"):
            doc.add_heading("Повестка дня", level=2)
        elif line.startswith("Цель встречи:") or line.startswith("* краткое описание цели встречи:"):
            # Добавляем описание цели встречи
            doc.add_paragraph(line, style='Normal')
        elif line.startswith("РЕЗЮМЕ ОБСУЖДЕНИЙ") or line.startswith("**I. Название вопроса**:"):
            doc.add_heading("Резюме обсуждений", level=2)
        elif line.startswith("Решено:") or line.startswith("* **Решено:**"):
            # Добавляем решение
            doc.add_paragraph(line, style='Normal')
        else:
            doc.add_paragraph(line, style='Normal')
    
    doc.save(filename)
    
    end_time = time.time()
    doc_time = end_time - start_time
    print(f"Время формирования DOCX: {doc_time:.2f} секунд")


In [10]:
# Путь к конфигурации и загрузка моделей
PATH_TO_CONFIG = "config.yaml"
pipeline = load_pipeline_from_pretrained(PATH_TO_CONFIG)
pipeline = pipeline.to(torch.device('cuda'))
device = "cuda"
whisper_model = whisperx.load_model("large-v3", device)

# Токен для API Yandex GPT
token = "t1.9euelZrPyM7Nko7KxsqekY2MmcnNju3rnpWanciXl8ubnczOi8qejpnOlc_l9PdaEgxJ-e81NEOE3fT3GkEJSfnvNTRDhM3n9euelZqKm4qej4uVicbJlJmUyJnIlO_8xeuelZqKm4qej4uVicbJlJmUyJnIlA.GaSMeXkhw_T4UUt9d35OtfeswBjCZR3e8ejajkUNmCR3u3f9cHjalXTE0YCICtOvd606Kf9ZypbVKXdtkr4RAA"

# Путь к папке с аудиофайлами
audio_folder = "test/"
output_csv = "results.csv"

# Обработка всех аудиофайлов в папке
results = []
for audio_file in os.listdir(audio_folder):
    if audio_file.endswith(('.mp3', '.m4a', '.ogg', '.wav')):
        file_path = os.path.join(audio_folder, audio_file)
        
        # Транскрипция и диаризация
        file_name, full_text, num_speakers, transcription_time = process_audio_file(file_path, pipeline, whisper_model, device)
        
        # Обработка текста через Yandex GPT
        processed_text = process_text_with_yandex_gpt(full_text, token)
        
        if processed_text:
            # Создание DOCX файла с замером времени
            doc_filename = f"{file_name}.docx"
            create_doc_from_text(processed_text, doc_filename)
        
        # Сохранение результата
        results.append({
            'Наименование аудиозаписи': file_name,
            'Транскрибированный текст': full_text,
            'Число спикеров': num_speakers
        })

# Запись результата в CSV
results_df = pd.DataFrame(results)
results_df.to_csv(output_csv, sep=';', index=False)
print(f"Результаты записаны в {output_csv}")


Loading pyannote pipeline from config.yaml...


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.3.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../datasphere/project/model/pyannote_model_segmentation-3.0.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu118. Bad things might happen unless you revert torch to 1.x.
No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:01<00:00, 8.87MiB/s]
Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.3.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../tmp/xdg_cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu118. Bad things might happen unless you revert torch to 1.x.
Конвертация test/Встреча 7. Аудиозапись.mp3 в WAV
Диаризация Встреча 7. Аудиозапись.wav
Транскрипция Встреча 7. Аудиозапись.wav
Detected language: ru (1.00) in first 30s of audio...
Время транскрипции: 20.81 секунд
Время формирования DOCX: 0.11 секунд
Конвертация test/Встреча 3. Аудиозапись.mp3 в WAV
Диаризация Встреча 3. Аудиозапись.wav
Транскрипция Встреча 3. Аудиозапись.wav
Detected language: ru (1.00) in first 30s of audio...
Время транскрипции: 41.17 секунд
Время формирования DOCX: 0.08 секунд
Конвертация test/Встреча 1. Аудиозапись.m4a в WAV
Диаризация Встреча 1. Аудиозапись.wav
Транскрипция Встреча 1. Аудиозапись.wav
Detected language: ru (0.99) in first 30s of audio...
Время транскрипции: 54.62 секунд
Время формирования DOCX: 0.08 секу